# Pipeline for Training and Analysis of 20 2x100 FC models

## Imports and Hyperparameters

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Subset

from PIL import Image

from src.cifar.models import *
from src.util import split_train_val, test, train, save_model

import matplotlib.pyplot as plt

import numpy as np

import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# this should print 'cuda' if you are assigned a GPU
print(device)

train_batch_size = 100
test_batch_size = 100
n_epochs = 5
learning_rate = 1e-2
seed = 100
input_dim = 32*32*3
out_dim = 10
num_hidden_layers = 2
layer_size = 100
momentum = 0.9
weight_decay_lam = 0.0001

# fc_model_params = [(2,100)]*20
fc_model_params = []

complex_models = [
    "cifar10_resnet20",
    "cifar10_resnet44",
    "cifar10_vgg11_bn",
    "cifar10_vgg16_bn",
    "cifar10_vgg19_bn",
    "cifar10_mobilenetv2_x0_5",
    "cifar10_shufflenetv2_x0_5",
    "cifar10_shufflenetv2_x2_0",
    "cifar10_repvgg_a0",
    "cifar10_repvgg_a2"
]

cuda


## Load Data

In [2]:
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
normalize_only = torchvision.transforms.Compose([torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
train_dataset = torchvision.datasets.CIFAR10('./datasets/', train=True, download=True, transform=transforms)

print(train_dataset.data.shape)

# get data from numpy files in ./datasets/cifar-10-1
test_data = np.load('./datasets/cifar-10-1/cifar10.1_v4_data.npy')
test_labels = np.load('./datasets/cifar-10-1/cifar10.1_v4_labels.npy')

# get means
means = np.mean(test_data, axis=(1,2,3), keepdims=True)
stds = np.std(test_data, axis=(1,2,3), keepdims=True)

# normalize data
test_data = (test_data - means)/stds

# create tensor
test_data = torch.from_numpy(test_data).float()
test_labels = torch.from_numpy(test_labels).long()

# create dataset 
test_dataset = torch.utils.data.TensorDataset(test_data, test_labels)

# sanity check
print('training data size:{}'.format(len(train_dataset)))
print('test data size:{}'.format(len(test_dataset)))

Files already downloaded and verified
(50000, 32, 32, 3)
training data size:50000
test data size:2021


In [3]:
train_dataset, val_dataset = split_train_val(train_dataset, valid_ratio=1/6)
print('training data size:{}'.format(len(train_dataset)))
print('validation data size:{}'.format(len(val_dataset)))

training data size:41667
validation data size:8333


## Loaders

In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=train_batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)
raw_test_loader = torch.utils.data.DataLoader(test_data, batch_size=test_batch_size, shuffle=False)

# sanity check
print('training data size:{}'.format(len(train_loader.dataset)))
print('validation data size:{}'.format(len(val_loader.dataset)))
print('test data size:{}'.format(len(test_loader.dataset)))

training data size:41667
validation data size:8333
test data size:2021


## Create Networks

In [5]:
fc_models = []  # [FC(input_dim, out_dim, num_hidden_layers, layer_size) for num_hidden_layers, layer_size in fc_model_params]
complex_models = load_pretrained_models(complex_models)

Using cache found in /home/willi/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master
Using cache found in /home/willi/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar10_resnet44-2a3cabcb.pt" to /home/willi/.cache/torch/hub/checkpoints/cifar10_resnet44-2a3cabcb.pt
100.0%
Using cache found in /home/willi/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master
Using cache found in /home/willi/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/vgg/cifar10_vgg16_bn-6ee7ea24.pt" to /home/willi/.cache/torch/hub/checkpoints/cifar10_vgg16_bn-6ee7ea24.pt
40.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_r

## Train Networks

In [6]:
for i, model in enumerate(fc_models):
    model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay_lam)

    print("Training FC model {}".format(i+1))

    for epoch in range(1, n_epochs + 1):
        train(model, train_loader, optimizer, epoch, device)

    test(model, val_loader, device)

    print("Saving FC model: {}".format(model))
    save_model(model, dataset="CIFAR10", filename="FC" + str(i))

"""for i, model in enumerate(os.listdir("./models/CIFAR10/")):
    if model.endswith(".pth") and "FC" in model:
        fc_models[i].load_state_dict(torch.load("./models/CIFAR10/" + model))
        with torch.no_grad():
            test(fc_models[i].to(device), test_loader, device)"""

'for i, model in enumerate(os.listdir("./models/CIFAR10/")):\n    if model.endswith(".pth") and "FC" in model:\n        fc_models[i].load_state_dict(torch.load("./models/CIFAR10/" + model))\n        with torch.no_grad():\n            test(fc_models[i].to(device), test_loader, device)'

## Missclassification Tracking

Images tracked as tuples (batch_idx, image_idx) and can be accessed from dataset as test_dataset[batch_idx][image_idx]

In [7]:
misses = dict()
complex_list = list(complex_models.values())
for model in fc_models + complex_list:
    print("Starting model test: {}".format(model))

    model.to(device)
    model.eval()

    with torch.no_grad():
        
        for i, (data, target) in enumerate(test_loader):

            data = data.permute(0,3,1,2).to(device)
            target = target.to(device)

            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)

            missed = pred.not_equal(target.data.view_as(pred)).view(-1).cpu().numpy()

            for j, miss in enumerate(missed):
                if miss:
                    if (i, j) in misses:
                        misses[(i, j)] += 1
                    else:
                        misses[(i, j)] = 1

    print("Finished")

Starting model test: CifarResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1),

In [8]:
all_miss = {k: v for k, v in misses.items() if v > 0}
significant = {k: v for k, v in misses.items() if v > 19}
print(len(significant))
print(len(all_miss))

0
812


In [9]:
y = list(all_miss.values()) + [0]*(len(test_dataset)-len(all_miss))

fig = plt.figure(figsize=(12, 8), facecolor="w")
plt.hist(y, [x-0.5 for x in range(22)], edgecolor="k")
plt.xlabel("Number of Missclassifications")
plt.ylabel("Frequency")
plt.savefig("plots/CIFAR10-1/im_freq_pretrained.png")
plt.close()

In [10]:
"""for sample in significant:

    idx = sample[0]*test_batch_size + sample[1]
    im_net = test_loader.dataset[idx][0].view(3, 32, 32)
    im_raw = raw_test_loader.dataset[idx][0].view(3, 32, 32)
    label = test_loader.dataset[idx][1]

    preds = np.array([model(im_net.unsqueeze(0).to(device)).argmax(dim=1, keepdim=True).cpu().numpy() for model in fc_models + complex_list]).flatten()
    fig = plt.figure(figsize=(12, 8), facecolor="w")
    plt.hist(preds, [x-0.5 for x in range(11)], edgecolor="k")
    plt.xlabel("Predicted Label")
    plt.ylabel("Frequency")
    plt.title("True Label: {}".format(label))
    plt.savefig("plots/CIFAR/images/im_" + str(idx) + "_freq.png")
    plt.close()

    fig = plt.figure()
    plt.imshow(im_raw.squeeze().cpu().numpy().transpose((1,2,0)))
    plt.title("Actual: {}".format(test_loader.dataset.targets[idx]))
    plt.savefig("plots/CIFAR/images/im_" + str(idx) + ".png")
    plt.close()"""

'for sample in significant:\n\n    idx = sample[0]*test_batch_size + sample[1]\n    im_net = test_loader.dataset[idx][0].view(3, 32, 32)\n    im_raw = raw_test_loader.dataset[idx][0].view(3, 32, 32)\n    label = test_loader.dataset[idx][1]\n\n    preds = np.array([model(im_net.unsqueeze(0).to(device)).argmax(dim=1, keepdim=True).cpu().numpy() for model in fc_models + complex_list]).flatten()\n    fig = plt.figure(figsize=(12, 8), facecolor="w")\n    plt.hist(preds, [x-0.5 for x in range(11)], edgecolor="k")\n    plt.xlabel("Predicted Label")\n    plt.ylabel("Frequency")\n    plt.title("True Label: {}".format(label))\n    plt.savefig("plots/CIFAR/images/im_" + str(idx) + "_freq.png")\n    plt.close()\n\n    fig = plt.figure()\n    plt.imshow(im_raw.squeeze().cpu().numpy().transpose((1,2,0)))\n    plt.title("Actual: {}".format(test_loader.dataset.targets[idx]))\n    plt.savefig("plots/CIFAR/images/im_" + str(idx) + ".png")\n    plt.close()'

# Changes

- add 0 bin on hist
- try cifar10.1
- regularize FCs (remove overfit)
    - weight decay
    - dropout (?)